In [1]:
import json
import random
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
from tqdm import trange

from pprint import pprint

from sklearn.feature_extraction.text import TfidfVectorizer

import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn.functional as F

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    BertModel, RobertaModel,
    BertPreTrainedModel,
    AdamW, get_linear_schedule_with_warmup,
    TrainingArguments,
)
from datasets import (
    Dataset,
    load_from_disk,
    concatenate_datasets,
)

from typing import List

In [2]:
# 난수 고정
def set_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # if use multi-GPU
    random.seed(random_seed)
    np.random.seed(random_seed)
    
set_seed(42) # magic number :)

In [3]:
print ("PyTorch version:[%s]."%(torch.__version__))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print ("device:[%s]."%(device))

PyTorch version:[1.6.0].
device:[cuda:0].


In [4]:
# Anwer
class DenseRetrieval:
    def __init__(self,
        args,
        dataset,
        tokenizer,
        p_encoder,
        q_encoder
    ):
        """
        학습과 추론에 사용될 여러 셋업을 마쳐봅시다.
        """

        self.args = args
        self.dataset = dataset

        self.tokenizer = tokenizer
        self.p_encoder = p_encoder
        self.q_encoder = q_encoder

    def train(self, args=None, tokenizer = None):
        if args is None:
            args = self.args
        if tokenizer is None :
            tokenizer = self.tokenizer

        q_seqs = tokenizer(self.dataset['question'], padding="max_length", truncation=True, return_tensors='pt')
        p_seqs = tokenizer(self.dataset['context'], padding="max_length", truncation=True, return_tensors='pt')

        train_dataset = TensorDataset(p_seqs['input_ids'], p_seqs['attention_mask'], p_seqs['token_type_ids'], 
                        q_seqs['input_ids'], q_seqs['attention_mask'], q_seqs['token_type_ids'])
        train_dataloader = DataLoader(train_dataset, batch_size=args.per_device_train_batch_size)

        no_decay = ["bias" ,"LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {"params": [p for n, p in self.p_encoder.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": args.weight_decay},
            {"params": [p for n, p in self.p_encoder.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
            {"params": [p for n, p in self.q_encoder.named_parameters() if not any(nd in n for nd in no_decay)], "weight_decay": args.weight_decay},
            {"params": [p for n, p in self.q_encoder.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0}
        ]
        optimizer = AdamW(
            optimizer_grouped_parameters,
            lr=args.learning_rate,
            # eps=args.adam_epsilon
        )

        t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
        scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total)
        
        global_step = 0

        self.p_encoder.zero_grad()
        self.q_encoder.zero_grad()
        torch.cuda.empty_cache()

        train_iterator = trange(int(args.num_train_epochs), desc="Epoch")

        for epoch, _ in enumerate(train_iterator):
            epoch_iterator = tqdm(train_dataloader, desc="Iteration")
            # loss_value=0 # Accumulation할 때 진행
            losses = 0
            for step, batch in enumerate(epoch_iterator):
                self.q_encoder.train()
                self.p_encoder.train()
            
                if torch.cuda.is_available():
                    batch = tuple(t.cuda() for t in batch)

                p_inputs = {'input_ids': batch[0],
                            'attention_mask': batch[1],
                            'token_type_ids': batch[2]
                            }
                
                q_inputs = {'input_ids': batch[3],
                            'attention_mask': batch[4],
                            'token_type_ids': batch[5]}
            
                p_outputs = self.p_encoder(**p_inputs)  # (batch_size, emb_dim)
                q_outputs = self.q_encoder(**q_inputs)  # (batch_size, emb_dim)

                # Calculate similarity score & loss
                sim_scores = torch.matmul(q_outputs, torch.transpose(p_outputs, 0, 1))  # (batch_size, emb_dim) x (emb_dim, batch_size) = (batch_size, batch_size)

                # target: position of positive samples = diagonal element 
                targets = torch.arange(0, args.per_device_train_batch_size).long()
                if torch.cuda.is_available():
                    targets = targets.to('cuda')

                sim_scores = F.log_softmax(sim_scores, dim=1)

                loss = F.nll_loss(sim_scores, targets)
                losses += loss.item()
                if step % 100 == 0 :
                    print(f'{epoch}epoch loss: {losses/(step+1)}') # Accumulation할 경우 주석처리

                loss.backward()
                #################ACCUMULATION###############################
                # loss_value += loss
                # if (step+1) % args.gradient_accumulation_steps == 0 :
                #     optimizer.step()
                #     scheduler.step()
                #     self.q_encoder.zero_grad()
                #     self.p_encoder.zero_grad()
                #     global_step += 1
                #     print(loss_value/args.gradient_accumulation_steps)
                #     loss_value = 0
                ############################################################
                optimizer.step()
                scheduler.step()
                self.q_encoder.zero_grad()
                self.p_encoder.zero_grad()
                global_step += 1
                
                torch.cuda.empty_cache()
                del p_inputs, q_inputs



In [5]:
class BertEncoder(BertPreTrainedModel):
    def __init__(self, config):
        super(BertEncoder, self).__init__(config)

        self.bert = BertModel(config)
        self.init_weights()
      
    def forward(
            self,
            input_ids, 
            attention_mask=None,
            token_type_ids=None
        ): 

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        
        pooled_output = outputs[1]
        return pooled_output

In [6]:
dataset = load_from_disk('/opt/ml/data/train_dataset')
train_dataset = dataset['train']

In [7]:
args = TrainingArguments(
    output_dir="dense_retireval",
    evaluation_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    weight_decay=0.01
)
model_checkpoint = "klue/bert-base"

# 혹시 위에서 사용한 encoder가 있다면 주석처리 후 진행해주세요 (CUDA ...)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
p_encoder = BertEncoder.from_pretrained(model_checkpoint).to(args.device)
q_encoder = BertEncoder.from_pretrained(model_checkpoint).to(args.device)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.bi

In [8]:
# Retriever는 아래와 같이 사용할 수 있도록 코드를 짜봅시다.
retriever = DenseRetrieval(
    args=args,
    dataset=train_dataset,
    tokenizer=tokenizer,
    p_encoder=p_encoder,
    q_encoder=q_encoder
)
retriever.train()

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

0epoch loss: 45.39480209350586
0epoch loss: 7.887749570431096
0epoch loss: 6.570167403909104


Epoch: 100%|██████████| 1/1 [05:30<00:00, 330.29s/it]

In [9]:
with open('/opt/ml/data/wikipedia_documents.json', "r", encoding="utf-8") as f:
    wiki = json.load(f)

corpus = list(
    dict.fromkeys([v["text"] for v in wiki.values()])
)  # set 은 매번 순서가 바뀌므로

In [10]:
p_encoder = retriever.p_encoder
q_encoder = retriever.q_encoder
with torch.no_grad() :
    p_encoder.eval()

    p_embs = []
    for p in tqdm(corpus) :
        p = tokenizer([p], padding='max_length', truncation=True, return_tensors='pt').to('cuda')
        p_emb = p_encoder(**p).to('cpu').numpy()
        p_embs.append(p_emb)
p_embs = torch.Tensor(p_embs).squeeze()

In [36]:
p_embs[3]

tensor([ 0.9143,  0.0093,  0.6034,  1.0000,  0.7705, -0.1520,  1.0000,  0.5420,
        -0.9997,  0.9789, -0.7481, -1.0000, -0.6572, -0.9847,  0.0947,  1.0000,
         0.9999, -0.9969, -1.0000,  0.6772,  0.9442,  1.0000, -0.8610,  0.9957,
        -0.7710, -0.9560,  1.0000,  1.0000,  1.0000,  1.0000,  0.8914,  1.0000,
        -0.9265, -1.0000,  0.9758, -0.9938,  1.0000,  1.0000,  0.7100, -0.2801,
         0.7355, -0.4112,  1.0000, -1.0000, -1.0000,  0.5649,  1.0000,  1.0000,
         0.9698, -0.8235, -1.0000, -0.9866, -0.9917,  0.8870,  0.8346,  0.8470,
        -1.0000,  1.0000, -0.6939, -0.9981,  0.9993, -0.2256, -0.9755, -0.7413,
        -0.9865, -0.8644,  0.1050, -0.9537, -1.0000, -0.7533, -0.9895,  0.9980,
        -0.8461, -0.9776, -0.7893,  0.3740,  0.9431, -0.9999,  0.4345,  0.9897,
        -0.9549,  0.7058, -0.9390,  0.9786,  0.9999,  1.0000,  0.9969, -1.0000,
        -0.7620, -0.5176,  1.0000, -0.9876, -0.8812, -0.9652, -0.5312, -0.9905,
         0.9923, -1.0000, -1.0000,  0.77

In [31]:
tmp = tokenizer([corpus[3]], padding = 'max_length', truncation=True, return_tensors='pt').to('cuda')
tmp_emb = p_encoder(**tmp).to('cpu').detach().numpy()
tmp_emb.shape

(1, 768)

In [12]:
corpus[0]

'이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.'

In [11]:
p_embs

tensor([[ 0.9144,  0.0094,  0.6034,  ..., -0.4013,  0.8149, -0.9985],
        [ 0.9143,  0.0093,  0.6033,  ..., -0.4012,  0.8148, -0.9985],
        [ 0.9143,  0.0094,  0.6033,  ..., -0.4011,  0.8148, -0.9985],
        ...,
        [ 0.9144,  0.0094,  0.6035,  ..., -0.4010,  0.8148, -0.9985],
        [ 0.9144,  0.0091,  0.6034,  ..., -0.4011,  0.8149, -0.9985],
        [ 0.9144,  0.0093,  0.6036,  ..., -0.4013,  0.8148, -0.9985]])

In [79]:
check = []
for i, aa in enumerate(corpus) :
    a = tokenizer(aa, padding='max_length', truncation=True, return_tensors='pt').to('cuda')
    check.append(a)
    if i == 2 :
        break

In [76]:
check[0]['input_ids'][0][:10]

tensor([    2,  1504,  6953,  2259,  3779, 10188,  2052,  2307,    16,  1537],
       device='cuda:0')

In [77]:
check[1]['input_ids'][0][:10]

tensor([    2,  1504, 10188,  2170, 11381,  3728,  3872,  2073, 20998,  2440],
       device='cuda:0')

In [55]:
def get_relavant_doc(query, q_encoder, p_embs, k=1) :

    with torch.no_grad() :
        q_encoder.eval()
        
        q_seqs_val = tokenizer(
                    [query],
                    padding="max_length",
                    truncation=True,
                    return_tensors="pt"
        ).to(args.device)
        q_emb = q_encoder(**q_seqs_val).to("cpu")  # (num_query=1, emb_dim)

    dot_prod_scores = torch.matmul(q_emb, torch.transpose(p_embs, 0, 1))
    rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()

    return dot_prod_scores, rank[:k]

In [65]:
p_embs

tensor([[-0.9920, -0.9073, -0.9912,  ...,  0.1764, -0.9692,  0.9948],
        [-0.9920, -0.9073, -0.9912,  ...,  0.1764, -0.9692,  0.9948],
        [-0.9920, -0.9073, -0.9912,  ...,  0.1764, -0.9692,  0.9948],
        ...,
        [-0.9920, -0.9073, -0.9912,  ...,  0.1764, -0.9692,  0.9947],
        [-0.9920, -0.9073, -0.9912,  ...,  0.1763, -0.9692,  0.9948],
        [-0.9920, -0.9073, -0.9912,  ...,  0.1763, -0.9692,  0.9948]])

In [64]:
dataset['validation']['question'][1]

'스카버러 남쪽과 코보콘그 마을의 철도 노선이 처음 연장된 연도는?'

In [60]:
doc_scores, doc_indices = get_relavant_doc(dataset['validation']['question'][0], q_encoder, p_embs, k = 2)


In [61]:
doc_indices

tensor([51633, 52323])

In [52]:
def get_relavant_doc(queries: List, q_encoder, p_embs, k=1) :

    with torch.no_grad() :
        q_encoder.eval()
        q_embs = []
        for q in queries :
            q = tokenizer([q], padding='max_length', truncation=True, return_tensors='pt').to('cuda')
            q_emb = q_encoder(**q).to('cpu').numpy()
            q_embs.append(q_emb)
    q_embs = torch.Tensor(q_embs).squeeze()

    result = torch.matmul(q_embs, torch.transpose(p_embs, 0, 1))
    if not isinstance(result, np.ndarray) :
        result = result.cpu().detach().numpy()

    doc_scores = []
    doc_indices = []
    for i in range(result.shape[0]) :
        sorted_result = np.argsort(result[i, :][::-1])
        doc_scores.append(result[i, :][sorted_result].tolist()[:k])
        doc_indices.append(sorted_result.tolist()[:k])

    return result, doc_scores, doc_indices

In [53]:
total = []
result, doc_scores, doc_indices = get_relavant_doc(dataset['validation']['question'], q_encoder, p_embs, k = 2)


In [54]:
doc_indices

[[52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157],
 [52442, 36157

In [22]:
doc_indices

[[52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [35624],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [29703],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [35624],
 [52442],
 [29711],
 [52442],
 [52442],
 [52442],
 [49808],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [48782],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [49808],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],
 [52442],


In [12]:

for idx, example in enumerate(
    tqdm(dataset['validation'], desc="Sparse retrieval: ")
):
    tmp = {
        # Query와 해당 id를 반환합니다.
        "question": example["question"],
        "id": example["id"],
        # Retrieve한 Passage의 id, context를 반환합니다.
        "context_id": doc_indices[idx],
        "context": " ".join(  # 기존에는 ' '.join()
            [corpus[pid] for pid in doc_indices[idx]]
        ),
    }
    if "context" in example.keys() and "answers" in example.keys():
        # validation 데이터를 사용하면 ground_truth context와 answer도 반환합니다.
        tmp["original_context"] = example["context"]
        tmp["answers"] = example["answers"]
    total.append(tmp)
cqas = pd.DataFrame(total)

TypeError: get_relavant_doc() missing 2 required positional arguments: 'q_encoder' and 'p_embs'

In [ ]:
from datasets import (
    Sequence,
    Value,
    Features,
    Dataset,
    DatasetDict,
)
f = Features(
    {
        "answers": Sequence(
            feature={
                "text": Value(dtype="string", id=None),
                "answer_start": Value(dtype="int32", id=None),
            },
            length=-1,
            id=None,
        ),
        "context": Value(dtype="string", id=None),
        "id": Value(dtype="string", id=None),
        "question": Value(dtype="string", id=None),
    }
)
datasets = DatasetDict({"validation": Dataset.from_pandas(df, features=f)})

## Reader